# Breast Cancer Classifier Comparison

By Iván del Horno Sotillo

This notebook is part of a Bachelor Thesis project. The main goal of this project is to compare different machine learning algorithms in order to find the best one for the classification of breast cancer. The dataset used is the [Breast Cancer Wisconsin (Diagnostic) Data Set](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)) from the UCI Machine Learning Repository.

The dataset contains 569 samples of benign and malignant tumor cells with features computed from an image obtained from a FNAB (Fine Needle Aspirate Biopsy).

More information will be available in the Thesis document.

## 1. Preparation

### Importing libraries and dataset

In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer

In [9]:
# Load the data
data = load_breast_cancer()

### Basic information about the dataset

In [10]:
print(data.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

Thanks to importing the dataset directly from Scikit-Learn, we can get some basic information about it.
We can see that we have 569 samples, out of which 212 are malignant and 357 are benign. The dataset has 30 features which are all numeric values. We can also see that there are no missing values.

The 30 features are based on 10 attributes (listed in the description above), and from each attribute we obtain the mean, the standard error and the worst (mean of the three largest values).

Then we have two classes, either malignant or benign tumor.

## 2. Exploratory Data Analysis

Now that we have loaded the dataset, and saw the basics of it, we can start a more in-depth exploration of the data.

This step is really important since it will help us understand the data that we are using, the relationships between the different features, which features are more important, etc. With this information we will be able to make more educated decisions when preprocessing the data and optimizing the classifiers.

To carry out this analysis we will do the following:
1. **Descriptive Statistics:** We will have a look at numerical description of the data by means of the measurements: mean, standard deviation, minimum, maximum, etc.
2. **Data Visualization:** We will plot the data in order to see the relationships between the different features and the classes.

### 2.1 Descriptive Statistics

#### Basic statistics

In [27]:
df = pd.DataFrame(data.data, columns=data.feature_names)

df.describe()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


From this quick statistical description we can see some interesting things:
- The median (50%) is usually a bit smaller than the mean, which would mean that the data is lightly skewed, we will see this in the next section.
- The difference between the 75% and the max is quite big in some cases (worst area 75% = 1084 and worst area max = 4254), which would mean that there are some outliers in the data, this will also be seen later.
- The range in some values is pretty large, for example in mean area we go from a min of 143.5 to a max of 2501.0, this means that we will have to scale the data in order to be able to use it in some classifiers.
- The standard deviation is also quite large in some cases, for example in worst area we have a standard deviation of 569.4, this points out that the data is really spread out emphasizing the need to scale the data.
- Lastly, some features have values that go from 0.05 to 0.16 and other from 185 to 4254, this further emphasizes the need to scale the data.

#### Skewness